In [1]:
import os
from tqdm import tqdm
import numpy as np
import tiktoken
from datasets import load_dataset # huggingface datasets

In [ ]:
# takes 5GB in huggingface .cache dir, about 727k documents
dataset = load_dataset("laiviet/wiki_727")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]